## Тема 13. Сегментация изображений с текстом:

### 1) Алгоритмы автоматического поворота изображения текста, чтобы текст был горизонтальным;

### 2) Удаление шума с изображения с текстом;

### 3) Нарезание изображения с текстом на символы, которые потом будут отправляться в нейронную сеть для распознавания.

In [1]:
from pytesseract import Output
import pytesseract
import imutils
import cv2
import numpy as np
from PIL import Image
import pytesseract

### 1) Алгоритмы автоматического поворота изображения текста, чтобы текст был горизонтальным;

In [23]:
import cv2
import numpy as np
import pytesseract

from PIL import Image

In [104]:
#почти рабочий чистый алгоритм Хафа

import math
import numpy as np

def hough_transform(image):
    # Dimensions of the image
    height, width = len(image), len(image[0])

    # List of detected lines. Each line is represented as a tuple (rho, theta).
    lines = []

    # Range of possible rho values
    rho_max = int((height**2 + width**2)**0.5)  # diagonal length of the image
    rho_values = np.linspace(-rho_max, rho_max, int(rho_max * 2.0))


    # Range of possible theta values (in degrees)
    theta_values = np.deg2rad(np.arange(0.0, 180.0, 0.5))  # step size of 0.5 degrees


    # Initialize the accumulator
    accumulator = np.zeros((len(rho_values), len(theta_values)))

    # For each pixel in the image
    for x in range(height):
        for y in range(width):
            # If the pixel is part of an edge
            if image[x][y] != 0:
                # For each possible theta value
                for theta_idx, theta in enumerate(theta_values):
                    # Calculate the corresponding rho value
                    rho = x * np.cos(theta) + y * np.sin(theta)
                    # Find the rho index that is closest to the calculated rho
                    rho_idx = np.argmin(np.abs(rho_values - rho))
                    # Increment the corresponding cell in the accumulator
                    accumulator[rho_idx][theta_idx] += 1

    # Find the maximum value in the accumulator
    max_value = np.max(accumulator)

    # For each cell in the accumulator
    for rho_idx in range(len(rho_values)):
        for theta_idx in range(len(theta_values)):
            # If the value in the cell is high enough
            if accumulator[rho_idx][theta_idx] > max_value * 0.5:  # threshold is 50% of the maximum value
                # Add the corresponding line to the list of detected lines
                lines.append((rho_values[rho_idx], np.rad2deg(theta_values[theta_idx])))

    return lines


In [107]:
hough_transform('Screenshot_5.png')

[(-0.5161290322580641, 90.5),
 (-0.5161290322580641, 91.0),
 (-0.5161290322580641, 91.5),
 (-0.5161290322580641, 92.0),
 (-0.5161290322580641, 92.5),
 (-0.5161290322580641, 93.0),
 (-0.5161290322580641, 93.5),
 (-0.5161290322580641, 94.0),
 (-0.5161290322580641, 94.5),
 (-0.5161290322580641, 95.0),
 (-0.5161290322580641, 95.5),
 (-0.5161290322580641, 96.00000000000001),
 (-0.5161290322580641, 96.5),
 (-0.5161290322580641, 97.0),
 (0.5161290322580641, 83.5),
 (0.5161290322580641, 84.0),
 (0.5161290322580641, 84.5),
 (0.5161290322580641, 85.0),
 (0.5161290322580641, 85.5),
 (0.5161290322580641, 86.0),
 (0.5161290322580641, 86.5),
 (0.5161290322580641, 87.0),
 (0.5161290322580641, 87.5),
 (0.5161290322580641, 88.0),
 (0.5161290322580641, 88.5),
 (0.5161290322580641, 89.0),
 (0.5161290322580641, 89.5),
 (0.5161290322580641, 90.0)]

In [77]:
# рисует одну линию, определяющую направление наклона текста

import cv2
import numpy as np

# Load the image
img = cv2.imread('Screenshot_2.png')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply edge detection method
edges = cv2.Canny(gray, 50, 150, apertureSize = 3)

# Apply Hough Line Transform
lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

# Iterate over the output lines and draw them on the image
for rho, theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 10000*(-b))
    y1 = int(y0 + 10000*(a))
    x2 = int(x0 - 10000*(-b))
    y2 = int(y0 - 10000*(a))

    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

# Save the image
cv2.imwrite('houghlines.jpg', img)

True

In [85]:
from PIL import Image
import cv2
import numpy as np

# Load the image
image = cv2.imread("Screenshot_1.png")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply edge detection method
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Apply Hough Line Transform
lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=200)

# Calculate line length based on image dimensions
height, width = image.shape[:2]
line_length = int(np.sqrt(height**2 + width**2))

# Draw the lines on the image
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + line_length * (-b))
    y1 = int(y0 + line_length * (a))
    x2 = int(x0 - line_length * (-b))
    y2 = int(y0 - line_length * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

# Display the image
cv2.imshow("Text Lines", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [121]:
# рисовка кучи линий 

from PIL import Image
import cv2
import numpy as np
    
def indicar_lines(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply edge detection method
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=200)

    # Calculate line length based on image dimensions
    height, width = image.shape[:2]
    line_length = int(np.sqrt(height**2 + width**2))

    # Draw the lines on the image
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + line_length * (-b))
        y1 = int(y0 + line_length * (a))
        x2 = int(x0 - line_length * (-b))
        y2 = int(y0 - line_length * (a))
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Calculate and print the slope in degrees
        slope_degrees = theta * (180 / np.pi)
        print(f"Slope of line: {slope_degrees} degrees")
    
    output = 'hugeline_' + image_path
    
    
    cv2.imwrite(output, image)

In [71]:
from PIL import Image
import cv2
import numpy as np
import math

def rotate_image(image_input):
    # Load the image
    image = cv2.imread(image_input)
    source = cv2.imread(image_input)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply edgedd detection method
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=200)

    # Calculate line length based on image dimensions
    height, width = image.shape[:2]
    line_length = int(np.sqrt(height**2 + width**2))

    # Initialize a variable to accumulate the sum of all slope angles
    total_degrees = 0

    # Draw the lines on the image
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + line_length * (-b))
        y1 = int(y0 + line_length * (a))
        x2 = int(x0 - line_length * (-b))
        y2 = int(y0 - line_length * (a))
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Calculate the slope in degrees and add it to the total
        slope_degrees = theta * (180 / np.pi)
        total_degrees += slope_degrees

    # Calculate the average slope
    average_slope_degrees = total_degrees / len(lines)
    print(f"Average slope: {average_slope_degrees} degrees")

    # Calculate the new image size
    diagonal = int(math.sqrt(height**2 + width**2))
    new_height = int(diagonal)
    new_width = int(diagonal)

    # Create a new image with white background
    new_image = np.ones((new_height, new_width, 3), dtype=np.uint8)
    new_image.fill(255)

    # Paste the source image into the center of the new image
    x_offset = (new_width - width) // 2
    y_offset = (new_height - height) // 2
    new_image[y_offset:y_offset+height, x_offset:x_offset+width] = source

    # Get the new center
    new_center = (new_width // 2, new_height // 2)

    # Get the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(new_center, average_slope_degrees-90, 1)

    # Perform the rotation
    rotated_image = cv2.warpAffine(new_image, rotation_matrix, (new_width, new_height))
    
    edit_image = 'rotated_' + image_input 
    
    # Save the image
    cv2.imwrite(edit_image, rotated_image)


In [72]:
rotate_image("Screenshot_1.png")

Average slope: 110.00537478448256 degrees


In [114]:
from PIL import Image
import cv2
import numpy as np
import math
import numpy as np

def rotate_image(image_input):
    # Load the image
    image = cv2.imread(image_input)
    source = cv2.imread(image_input)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply edge detection method
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=200)

    # Calculate line length based on image dimensions
    height, width = image.shape[:2]
    line_length = int(np.sqrt(height**2 + width**2))

    # Initialize a list to hold all slope angles
    slope_degrees_list = []

    # Draw the lines on the image
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + line_length * (-b))
        y1 = int(y0 + line_length * (a))
        x2 = int(x0 - line_length * (-b))
        y2 = int(y0 - line_length * (a))
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Calculate the slope in degrees and add it to the list
        slope_degrees = theta * (180 / np.pi)
        slope_degrees_list.append(slope_degrees)

    # Calculate the IQR
    Q1 = np.percentile(slope_degrees_list, 25)
    Q3 = np.percentile(slope_degrees_list, 75)
    IQR = Q3 - Q1

    # Filter out the outliers
    filtered_slope_degrees_list = [x for x in slope_degrees_list if (Q1 - 1.5 * IQR < x < Q3 + 1.5 * IQR)]

    # Calculate the average slope
    average_slope_degrees = np.mean(filtered_slope_degrees_list)
    print(f"Average slope: {average_slope_degrees} degrees")

    # Calculate the new image size
    diagonal = int(math.sqrt(height**2 + width**2))
    new_height = int(diagonal)
    new_width = int(diagonal)

    # Create a new image with white background
    new_image = np.ones((new_height, new_width, 3), dtype=np.uint8)
    new_image.fill(255)

    # Paste the source image into the center of the new image
    x_offset = (new_width - width) // 2
    y_offset = (new_height - height) // 2
    new_image[y_offset:y_offset+height, x_offset:x_offset+width] = source

    # Get the new center
    new_center = (new_width // 2, new_height // 2)

    # Get the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(new_center, average_slope_degrees-90, 1)

    # Perform the rotation
    rotated_image = cv2.warpAffine(new_image, rotation_matrix, (new_width, new_height))
    
    edit_image = 'rotated_' + image_input 
    
    # Save the image
    cv2.imwrite(edit_image, rotated_image)


In [150]:
rotate_image("editA_photo_2023-07-06_23-09-16.jpg")


Average slope: 9.232558164810207 degrees


In [155]:
rotate_image('Screenshot_4.png')

Average slope: 9.294117688062634 degrees


In [83]:
rotate_image("Screenshot_5.png")
rotate_image("Screenshot_6.png")

Average slope: 66.08064499883295 degrees
Average slope: 122.571431006548 degrees


In [122]:
indicar_lines("Screenshot_5.png")

Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 65.00000067042492 degrees
Slope of line: 65.00000067042492 degrees
Slope of line: 65.00000067042492 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 65.00000067042492 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 65.00000067042492 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 65.00000067042492 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 65.00000067042492 degrees
Slope of line: 66.00000183661733 degrees
Slope of line: 6

In [123]:
indicar_lines("Screenshot_6.png")

Slope of line: 123.00000000769224 degrees
Slope of line: 123.00000000769224 degrees
Slope of line: 123.00000000769224 degrees
Slope of line: 123.00000000769224 degrees
Slope of line: 122.00000567168901 degrees
Slope of line: 123.00000000769224 degrees
Slope of line: 123.00000000769224 degrees
Slope of line: 123.00000000769224 degrees
Slope of line: 122.00000567168901 degrees
Slope of line: 123.00000000769224 degrees
Slope of line: 122.00000567168901 degrees
Slope of line: 122.00000567168901 degrees
Slope of line: 122.00000567168901 degrees
Slope of line: 122.00000567168901 degrees


In [148]:
indicar_lines("editC_photo_2023-07-06_23-09-16.jpg")

Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.99999970909292 degrees
Slope of line: 9.000000250447817 degrees
Slope of line: 9.0000002

### 2) Удаление шума с изображения с текстом

To remove digital noise from images with text, there are several methods and tools available. Here are some steps and considerations to help you achieve this:

**Image Smoothing**: One approach to removing noise from images is by applying image smoothing techniques. This involves reducing the high-frequency components in the image, which are often associated with noise. You can adjust the degree of smoothing to control the intensity of the noise removal. However, be cautious when setting high smoothing values, as it may result in an unnatural, plastic-like appearance for brightness noise or color changes for color noise.

**Noise Suppression**: On grayscale images, where there is no color component, you only need to adjust the parameters for brightness noise suppression. The histogram of the image can help you determine which parameter to modify for the best results. It characterizes the noise level of the original image and remains unchanged when adjusting the settings on the control panel.

**Quality and Noise Level**: Some tools provide additional parameters such as quality and noise level. The quality parameter reduces the number of color spots during noise smoothing, but it significantly increases processing time. The noise level parameter determines what to consider as noise (brightness or color) and what to treat as important image details. Be cautious with high noise level values, as they may result in the smoothing of fine image details.

**Sensor Level Noise Suppression**: To prevent or reduce digital noise at the sensor level, techniques such as using larger pixels and micro-lenses that are closely packed together can be employed. Color filters that allow a higher percentage of light can also be used. However, it's important to note that using color filters may negatively impact color reproduction.

**Exposure Time**: Longer exposure times can contribute to higher levels of noise, especially due to the increase in thermal noise from the electronic components. To mitigate this, consider reducing the exposure time while maintaining the same amount of light reaching the sensor.

**Interpolation**: Interpolation methods are used to handle defective pixels in image sensors. These methods replace the defective pixel with neighboring pixels or calculate a value based on surrounding elements. While interpolation helps mitigate the impact of defective pixels, it can affect image sharpness and introduce artifacts.

**Post-processing**: Digital noise can be further reduced through post-processing techniques. One common approach is to average the pixel brightness within a group of pixels that are considered similar. However, this can result in a loss of image detail and a "softer" appearance. It's important to note that false details may also appear in the image due to the algorithm's interpretation of similarity. Careful parameter tuning is necessary to achieve the desired noise reduction without sacrificing image quality.

#### solution using the OpenCV and the PyTesseract library

Method 1: Background Correction

In [140]:
import cv2

# Load the image
def noise_removeA(im):
    image = cv2.imread(im)
    # Convert the image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Define the structuring element
    se = cv2.getStructuringElement(cv2.MORPH_RECT , (8,8))
    # Dilate the image
    bg = cv2.morphologyEx(image, cv2.MORPH_DILATE, se)
    # Divide the image by the background
    out_gray = cv2.divide(image, bg, scale=255)
    # Apply Otsu's thresholding
    out_binary = cv2.threshold(out_gray, 0, 255, cv2.THRESH_OTSU)[1]
    cv2.imwrite('editA_' + im,  out_binary)

In [141]:
noise_removeA("photo_2023-07-06_23-09-16.jpg")

Method 2: Division Normalization

In [142]:
import cv2
import numpy as np

def noise_removeB(im):
    img = cv2.imread(im)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (0,0), sigmaX=33, sigmaY=33)

    # Divide the original image by the blurred image
    divide = cv2.divide(gray, blur, scale=255)

    # Apply Otsu's thresholding
    thresh = cv2.threshold(divide, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    # Define the structuring element
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    # Apply morphological operations
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    cv2.imwrite('editB_' + im,  morph)

In [143]:
noise_removeB("photo_2023-07-06_23-09-16.jpg")

In [146]:
def noise_removeC(im):
    import cv2

    # Load the image
    image = cv2.imread(im, 0)

    # Apply median filter
    filtered_image = cv2.medianBlur(image, 5)  # 5 is the kernel size

    # Save the result
    cv2.imwrite('editC_' + im, filtered_image)


In [147]:
noise_removeC('photo_2023-07-06_23-09-16.jpg')

### 3) Нарезание изображения с текстом на символы, которые потом будут отправляться в нейронную сеть для распознавания

In [ ]:
from PIL import Image, ImageDraw

def highlight_letters(image_path, output_path):
    image = Image.open(image_path).convert("L")
    pixels = image.load()


    marked_pixels = []
    for i in range(image.width):
        for j in range(image.height):
            if pixels[i, j] < 128:
                marked_pixels.append((i, j))


    groups = []
    current_group = [marked_pixels[0]]
    for pixel in marked_pixels[1:]:
        if abs(pixel[0] - current_group[-1][0]) <= 1 and abs(pixel[1] - current_group[-1][1]) <= 1:
            current_group.append(pixel)
        else:
            groups.append(current_group)
            current_group = [pixel]
    groups.append(current_group)

    draw = ImageDraw.Draw(image)
    for group in groups:
        min_x = min(pixel[0] for pixel in group)
        max_x = max(pixel[0] for pixel in group)
        min_y = min(pixel[1] for pixel in group)
        max_y = max(pixel[1] for pixel in group)
        draw.rectangle([(min_x, min_y), (max_x, max_y)], outline="red")

    image.save(output_path)


In [ ]:
highlight_letters("rotated_Screenshot_4.png")